### Dependencies and start up material

In [38]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo

In [39]:
# Add splinter depencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [40]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/Haley/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


## NASA Mars News

In [41]:
# URL of page to be scraped
url1 = 'https://mars.nasa.gov/news/'

In [42]:
# Retrieve page with the requests module
response1 = requests.get(url1)

In [43]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response1.text, 'html.parser')

In [44]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

pacer">
    </div>
    <div id="page">
     <div class="page_cover">
     </div>
     <!-- title to go in the page_header -->
     <div class="header_mask">
      <section class="content_page module">
      </section>
     </div>
     <div class="grid_list_page module content_page">
      <div class="grid_layout">
       <article>
        <header id="page_header">
        </header>
        <div class="react_grid_list grid_list_container" data-react-cache-id="GridListPage-0" data-react-class="GridListPage" data-react-props='{"left_column":false,"class_name":"","default_view":"list_view","model":"news_items","view_toggle":false,"search":"true","list_item":"News","title":"News","categories":["19,165,184,204"],"order":"publish_date desc,created_at desc","no_items_text":"There are no items matching these criteria.","site_title":"NASA’s Mars Exploration Program ","short_title":"Mars","site_share_image":"/system/site_config_values/meta_share_images/1_mars-nasa-gov.jpg","per_page":null,"filter

In [45]:
paragraph_result = soup.find_all('div', class_="rollover_description_inner")
paragraph_result = soup.find_all('div', class_="article_teaser_body")
paragraph_result
#paragraph_result[0].text.strip()

[]

In [46]:
# Collect the latest news title and paragraph text
title_result = soup.find_all(class_="content_title")

# Save them in variables
news_title = title_result[0].text.strip()
print(news_title)
#news_p = paragraph_result[0].text.strip()
#news_p

NASA Ingenuity Mars Helicopter Prepares for First Flight


## JPL Mars Space Images - Featured Image

In [47]:
# URL of page to be scraped
url2 = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url2)

In [48]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Save result of image
image_url = soup.find(class_="headerimage fade-in")['src']

# Assign the url string to a variable
featured_image_url = f'{url2}/{image_url}'
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html/image/featured/mars2.jpg


## Mars Facts

In [49]:
# URL of page to be scraped
url3 = 'https://space-facts.com/mars/'

In [50]:
# Grab tables from the website
tables = pd.read_html(url3)

In [51]:
# Grab the first table containing facts about the planet including Diameter, Mass, etc.
df = tables[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [52]:
# Clean Up DF
df = df.rename(columns={0:"Description"})
df = df.rename(columns={1:"Mars"})
df = df.set_index('Description')
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [53]:
# Use Pandas to convert the data to a HTML table string.
df.to_html('mars_facts.html')

## Mars Hemispheres

In [54]:
# URL of page to be scraped
url4 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url4)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [61]:
# 
sidebar = soup.find('div', class_='collapsible results')
items = sidebar.find_all('div', class_='item')
hemisphere_image_urls = []

In [70]:
for item in items:
    # Error handling
    try: 
        # Find the image title
        image = item.find('div', class_='description')
        title = image.h3.text
        # Find the image url
        image_url = image.a['href']
        base_url = 'https://astrogeology.usgs.gov'
        # Make connection to full resoltuion image by navigating to link
        browser.visit(base_url+image_url)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        image_src = soup.find('li').a['href']
        # Confirm results are being extracted
        print(title)
        print(image_src)
        print(' ')
        # Add title and image url to dict then add to list
        image_dict = {'title':title, 
                    'image_url':image_src}
        hemisphere_image_urls.append(image_dict)
    except Exception as e:
        print(e)

Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
 
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
 
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
 
Valles Marineris Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
 


### Quit Connection with Browser

In [71]:
# Quit browser
browser.quit()